<a href="https://colab.research.google.com/github/Ahirvoas/Training-Day2/blob/main/Tutorial_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wind Turbine Modeling Workshop

# DAY 2 - Case - Floating offfshore wind turbine steady wind

<img src="Pictures/OC3_FAST.png" width="150"/>

This tutorial is based on the developed model based on the Offshore Code Comparison 3 (model) which is a spar buoy structure with the NREL 5MW wind turbine atop.The gross properties chosen for the NREL 5-MW baseline wind turbine are gathered in the following table.

| Rating | 5 MW |
|:---:|:---:|
| Rotor orientation, Configuraiton | Upwind, 3 Blades |
| Rotor, Hub Diameter | 126 m, 3m |
| Hub Height | 90 m |
| Cut-In, Rated, Cut-Out Wind Speed | 3m/s, 11.4m/s, 25m/s |
| Cut-in, rated rotor speed | 6.9 rpm, 12.1 rpm |
| Rotor Mass | 110,000 kg |
| Nacelle Mass | 240,000 kg |
| Tower Mass | 347,460 kg |

In this tutorial, we will consider, hereafter, a **steady wind** as external sollicitation. 



In [ ]:
# Package pyfast importation  (and some useful packages)

from pyFAST.input_output import FASTInputFile
from pyFAST.case_generation.runner import run_fast
from pyFAST.input_output import FASTOutputFile
import matplotlib.pyplot as plt
from sys import platform

import numpy as np
from scipy import signal
import os
from scipy.integrate import simpson

## Test of the OpenFAST configuration on your PC

In [ ]:
# Specify the OpenFAST binaries and OpenFAST main file for simulation

filename_servo = './NRELOffshrBsline5MW_OC3Hywind_ServoDyn.dat' # Insert the name of the InflowWind file with the path (relative or absolute)
f_servo = FASTInputFile(filename_servo)
if platform == "linux" or platform == "linux2" or platform == "darwin":
    FAST_EXE  = '{}'.format(os.popen('which openfast').read())
    path = 'OffshoreSteadyWindCtrl-Linux'
else:
    FAST_EXE  = './openfast_x64.exe'
    path = 'OffshoreSteadyWindCtrl'
f_servo.write(filename_servo)
 # Location of a FAST exe (and dll) (For Linux/MAC users: FAST_EXE  = openfast)


main_file = './{}/NRELOffshrBsline5MW.fst'.format(path)  # Main file in OpenFAST directory

Run the OpenFAST simulation to test by running the function **run_fast** with main_file

In [ ]:
# INSERT CODE

## Run the floating offshore  wind turbine simulation

### 1- Modification of the Input Files

In practice, OpenFAST models the system via computational modules for aerodynamics, hydrodynamics for offshore structures, control and electrical (servo) system dynamics, and structural dynamics to enable coupled non-linear aero-hydro-servo-elastic simulation in the time domain.

<img src="Pictures/modules.png" width="500"/>

Each of these modules is independent of the others and is based on a configuration file that creates an arborescence of files to carry out an OpenFAST simulation.

<img src="Pictures/arborescence.png" width="500"/>

#### Modification of the simulation length

Change in the main file the total run time. You can use the function used in the previous cell named **FASTInputFile** in order to modify the file with the extension ".fst".

In [ ]:
# INSERT CODE

### 2- Run OpenFAST with the new modified files.

**Hint:**
    
    You can use the previous command used during the DAY 1 tutorial in order to run the simulation (the run_fast function developed in the pyFAST package).

In [ ]:
# INSERT CODE

#### Rotor speed and generated power of the wind turbine:

Plot the rotor speed and the generated power of the wind turbine versus the time obtained from the previous simulation. For comparison, you can plot the two time-series on the same graph thanks to the function *subplot*. 

Hint:

You can find the corresponding variables of the Pandas Dataframe for the electrical generator power by looking at the Excel file *OutListParameters.xlsx*. 

An example of the python function to be used:

```python
import numpy as np
import matplotlib.pyplot as plt


def f(t):
    return np.exp(-t) * np.cos(2*np.pi*t)


t1 = np.arange(0.0, 5.0, 0.02)

plt.figure()
plt.subplot(211)
plt.plot(t1, f(t1), color='black')

plt.subplot(212)
plt.plot(t1, np.cos(2*np.pi*t1), color='tab:orange', linestyle='--')
plt.show()
```

In [ ]:
# INSERT CODE

#### Heave displacement of the floating offshore wind turbine:

Plot the heave displacement of the wind turbine platform versus the time obtained from the previous simulation. Then you will have to see the spectral response of the wind turbine platform. You will consider a transient period of 200 seconds before to perform a spectral analysis.

<img src="Pictures/Degrees-of-freedom-floating-wind-turbine.jpg" width="500"/>

**Hint:**

    - Load the output file in a new pandas dataframe using df = FASTOutputFile(NAMEFILE).toDataFrame()
    
    - Use the welch method of the signal python package for the spectral analysis of the temporal series

Example of the welch method

```python
from scipy import signal
fs = 10e3
N = 1e5
amp = 2*np.sqrt(2)
freq = 1234.0
noise_power = 0.001 * fs / 2
time = np.arange(N) / fs
x = amp*np.sin(2*np.pi*freq*time)
x += rng.normal(scale=np.sqrt(noise_power), size=time.shape)
f, Pxx_den = signal.welch(x, fs, nperseg=1024)
```

For the exercise, we will consider:\
    - nperseg = 2^9\
    - maximum frequency = 1.5\
    - a transition time of 200 seconds

In [ ]:
# INSERT CODE FOR TEMPORAL REPRESENTATION

In [ ]:
# INSERT CODE FOR SPECTRAL ANALYSIS USING WELCH

In [ ]:
# INSERT CODE FOR SPECTRAL ANALYSIS PLOT

### 2 - Numerical modeling versus reality

A numerical model cannot be a perfect representation of the real asset. First, the numerical model is a mathematical representation of the system and consequently cannot represent perfectly the real behavior. Moreover, some errors can also appear in the modeling assumptions that we have made in order to build the numerical model.

In this section, we will investigate the effect of the second source of error due to a mispecification of the parameters in the numerical model.

**Exercice:**
    
    You will have to load the simulation named 'real_simulation.npy' of the OC3-Hywind floating wind turbine with an increase of 1% of the platform mass. This simulation will be considered as the response of the real system. Then, you will compare the spectral response in heave of the simulation representing the real system with the one performed at the beginning of this tutorial.

**Hint:**

    - Change the value of the platform mass in the ElastoDyn file
    - Use the function FASTInpputFile function
    - Use the run_fast function

#### Modification of the ElastoDyn input file

Open the excel file named **OutListParameters.xlsx** and find the variable linked to the platform mass. Then, modify the value of this variable in the Elastodyn file by increasing the default value of 1%.

In [ ]:
filename_elastodyn = r'./OffshoreSteadyCtrl/NRELOffshrBsline5MW_OC3Hywind_ElastoDyn.dat' # Insert the name of the InflowWind file with the path (relative or absolute)
f_elasto = FASTInputFile(filename_elastodyn)
f_elasto['PtfmMass'] = 7.46633E+06+ 1*7.46633E+06/100
new_filename_elastodyn = r'./OffshoreSteadyCtrl/NRELOffshrBsline5MW_OC3Hywind_ElastoDyn_real.dat' # Insert the name of the InflowWind file with the path (relative or absolute)
f_elasto.write(new_filename_elastodyn)

#### Post-treatment of the simulation


In [ ]:
# INSERT CODE

2- Power spectrum density

Analyze the spectral reponse of the wind turbine platform heave by using the Welch method. Hereafter, you will again consider a transient period of 200 seconds before to perfom a spectral analysis. 

In [ ]:
# INSERT CODE

3- Comparison with default simulation

Load the default simulation named **NRELOffshrBsline5MW_anomaly.out** located in the sub-folder *data*. Again, analyze the spectral response of the wind platform heave by using the welch method. A transcient period of 200 seconds will be considered.

In [ ]:
# INSERT CODE

## Question:
### - Which metric can you choose to compare the two spectral responses?
### - How to know if it's an anomaly or a mispecification of the numerical model?

In [ ]:
# INSERT CODE

## 3 - Identification of anomalies

1 - Load the npy file located in the folder data\
2 - Create a PSD for each signal in the dataset. Don't forget to not consider the transition time

In [ ]:
dataset = np.loadtxt(f'./{path}/data/dataset.txt')
# INSERT CODE

In [ ]:
# INSERT CODE TO PRINT NUMBER OF ANOMALIES